In [3]:
import cv2
import numpy as np

def augmented_image(frame,im_src, pts_src, pts_dst):
    
    # Calculate Homography
    h, status = cv2.findHomography(pts_src, pts_dst)

    # Warp source image to destination based on homography
    warped_image = cv2.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
            
    # Prepare a mask representing region to copy from the warped image into the original frame.
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
    im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
    im_out = cv2.add(im_out, warped_image)
    
    return im_out

def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)

qrcode = cv2.imread('./images/aruco_qrcode.png')
qrcode_size = qrcode.shape[:2]
qrcode_points = np.float32([[0,0], [qrcode_size[1],0],[qrcode_size[1], qrcode_size[0]] ,[0, qrcode_size[0]] ])

template_img = cv2.imread('./images/final_exam/Templates/Template-1.png')
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)
# qrcode = cv2.imread()

vid = cv2.VideoCapture('./videos/final_exam/Dataset-2/left_output.avi')

params_dir = os.getcwd()+'/Final1/'
print(params_dir)

#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

print("Camera matrix")
print(K)
print("Len distortion")
print(dist)

AruCo_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_1000)
AruCo_params = cv2.aruco.DetectorParameters_create()

while vid.isOpened() :
    ret, frame = vid.read()
    
    if ret :
        query_img = frame.copy()
        query_gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
        # detected =  feature_object_detection(template_img, template_gray, frame, frame_gray)
        template_kpts, template_desc = cv2.SIFT_create().detectAndCompute(template_gray, None)
        query_kpts, query_desc = cv2.SIFT_create().detectAndCompute(query_gray, None)
        matches = cv2.BFMatcher().knnMatch(template_desc, query_desc, k=2)
        good_matches = list()
        good_matches_list = list()
        for m, n in matches :
            if m.distance < 0.65*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])
        
        if len(good_matches) > 10:
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 0.9) # H RANSAC
            # get the bounding box around template image
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)

            detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (0,0,255), 3, cv2.LINE_AA)
            drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
            
            augmented = augmented_image(query_img, qrcode, qrcode_points, np.int32(transformed_box))

            markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(augmented, AruCo_dict, parameters = AruCo_params)

            if len(markerCorners) > 0:
                rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.1, K, dist)
                for (rvec, tvec, corner) in zip(rvecs, tvecs, markerCorners) :
                    x = tvec[0,0]
                    y = tvec[0,1]
                    z = tvec[0,2]
                    text = "x: {:.3f} y: {:.3f} z: {:.3f}".format(x, y, z)
                    cX = (corner[0,0][0] + corner[0,2][0]) / 2
                    cY = (corner[0,0][1] + corner[0,2][1]) / 2
                    write_text(detected_img, (cX, cY), 20, text)
            
            cv2.imshow('Video frame', detected_img)
            # cv2.imshow('Video frame_gray', drawmatch)
        else :
            print('Keypoints not enough')
            cv2.imshow('Video frame', frame)


        if cv2.waitKey(int(1000/30)) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()

d:\KMUTNB\ปี3\Image Processing\010723305-main/Final1/
Camera matrix
[[726.47630281   0.         466.2575863 ]
 [  0.         732.06406134 251.99786765]
 [  0.           0.           1.        ]]
Len distortion
[[ 0.02450981  0.03012141  0.00045661  0.01036187 -0.07354295]]
Keypoints not enough
